In [ ]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURAÇÃO: PESOS DA FÓRMULA ---
WEIGHTS = {
    'PSxG_Net': 1.0,         
    'Crosses_Stopped': 0.15, 
    'OPA': 0.10             
}

def load_data():
    path = "data/processed/pl_goalkeepers_2526_matches.csv"
    if not os.path.exists(path):
        raise FileNotFoundError(f"Arquivo não encontrado: {path}. Rode o step 01 primeiro.")
    return pd.read_csv(path)

def calculate_dynamic_sos(df):
    """
    Cria o Strength of Schedule baseado no perigo real (PSxG) que cada time gera.
    Retorna um dicionário { 'Nome do Time': Multiplicador }.
    """
    print("🤖 Calculando Nível de Ameaça dos Times (Data-Driven)...")
    
    # 1. Calcular quanto PSxG cada time gerou em média quando foi Oponente
    # Agrupamos pelo 'Opponent' (Time que chutou) e tiramos a média do PSxG que ele forçou
    offensive_stats = df.groupby('Opponent')['PSxG'].mean()
    
    # 2. Calcular a média da Liga inteira (A linha base)
    league_avg_psxg = offensive_stats.mean()
    
    # 3. Criar o Multiplicador (Ratio)
    # Ex: Se City gera 2.0 e a média é 1.5, o peso é 1.33x
    sos_factors = offensive_stats / league_avg_psxg
    
    # 4. Suavização (Dampening)
    # Para evitar que um time muito fraco tenha peso 0.2 ou um muito forte tenha peso 3.0
    # Fórmula: Peso Final = 1 + (Diferença * 0.5)
    # Isso comprime os valores para ficarem mais próximos de 1.0 (segurança estatística)
    sos_factors_dampened = 1 + (sos_factors - 1) * 0.5
    
    # Converter para dicionário para mapeamento rápido
    sos_map = sos_factors_dampened.to_dict()
    
    # Mostra os 3 times mais difíceis e os 3 mais fáceis para validação
    sorted_sos = sorted(sos_map.items(), key=lambda x: x[1], reverse=True)
    print(f"   🔥 Maior Ameaça: {sorted_sos[:3]}")
    print(f"   🧊 Menor Ameaça: {sorted_sos[-3:]}")
    
    return sos_map

def calculate_match_score(df, sos_map):
    print("🧮 Calculando Scores por Partida...")
    
    # 1. Score Base
    score_base = (
        (df['PSxG_Net'] * WEIGHTS['PSxG_Net']) +
        (df.get('Crosses_Stopped', 0) * WEIGHTS['Crosses_Stopped']) +
        (df.get('OPA', 0) * WEIGHTS['OPA'])
    )
    
    # 2. Aplicar SoS Dinâmico
    # Mapeia o oponente ao seu fator calculado. Se não achar, usa 1.0
    sos_multiplier = df['Opponent'].map(sos_map).fillna(1.0)
    
    df['Match_Score'] = score_base * sos_multiplier
    
    # Guardar o multiplicador no DF para podermos auditar depois se quisermos
    df['Opponent_Difficulty'] = sos_multiplier
    
    return df

def calculate_hybrid_ranking(df):
    print("📈 Calculando Ranking Híbrido (Forma + Consistência)...")
    
    df = df.sort_values(by=['Player', 'Date'])
    grouped = df.groupby('Player')['Match_Score']
    
    # Forma Recente (Last 5 Games)
    df['Form_Score'] = grouped.rolling(window=5, min_periods=1).mean().reset_index(0, drop=True)
    
    # Consistência (Temporada)
    df['Season_Score'] = grouped.expanding(min_periods=3).mean().reset_index(0, drop=True)
    
    # Score Final Ponderado
    df['Final_Score'] = (df['Form_Score'] * 0.6) + (df['Season_Score'] * 0.4)
    
    return df

def generate_leaderboard(df):
    latest = df.sort_values('Date').groupby('Player').tail(1)
    
    # Filtro: Mínimo 400 minutos jogados (aprox 4.5 jogos)
    valid_players = df.groupby('Player')['Minutes'].sum()
    valid_players = valid_players[valid_players >= 400].index
    
    leaderboard = latest[latest['Player'].isin(valid_players)].copy()
    
    cols = ['Player', 'Team', 'Final_Score', 'Form_Score', 'Season_Score', 'Match_Score']
    leaderboard = leaderboard[cols].sort_values(by='Final_Score', ascending=False)
    
    leaderboard['Rank'] = range(1, len(leaderboard) + 1)
    return leaderboard

if __name__ == "__main__":
    try:
        # 1. Carregar
        df = load_data()
        
        # 2. Calcular SoS Dinâmico (NOVO!)
        dynamic_sos_map = calculate_dynamic_sos(df)
        
        # 3. Calcular Scores
        df_scored = calculate_match_score(df, dynamic_sos_map)
        
        # 4. Ranking
        df_ranked = calculate_hybrid_ranking(df_scored)
        
        # 5. Exportar
        leaderboard = generate_leaderboard(df_ranked)
        
        df_ranked.to_csv("data/processed/pl_goalkeepers_full_metrics.csv", index=False)
        leaderboard.to_csv("data/processed/final_ranking_table.csv", index=False)
        
        print("\n🏆 TOP 5 GOLEIROS (SISTEMA DINÂMICO):")
        print(leaderboard[['Rank', 'Player', 'Team', 'Final_Score']].head(5))
        
    except Exception as e:
        print(f"❌ Erro: {e}")